In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('train.csv', parse_dates=['date'])
test = pd.read_csv('test.csv', parse_dates=['date'])
sample_sub = pd.read_csv('sample_submission.csv')

In [3]:
print('Train shape:{}, Test shape:{}'.format(train.shape, test.shape))

Train shape:(913000, 4), Test shape:(45000, 4)


In [4]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [5]:
print("number of unique items :",train.item.nunique())
print("number of unique store :",train.store.nunique())

number of unique items : 50
number of unique store : 10


In [6]:
train['dayofmonth'] = train.date.dt.day
train['dayofyear'] = train.date.dt.dayofyear
train['dayofweek'] = train.date.dt.dayofweek
train['month'] = train.date.dt.month
train['year'] = train.date.dt.year
train['weekofyear'] = train.date.dt.weekofyear

In [7]:
train.head()

,date,store,item,sales,dayofmonth,dayofyear,dayofweek,month,year,weekofyear
0,2013-01-01,1,1,13,1,1,1,1,2013,1
1,2013-01-02,1,1,11,2,2,2,1,2013,1
2,2013-01-03,1,1,14,3,3,3,1,2013,1
3,2013-01-04,1,1,13,4,4,4,1,2013,1
4,2013-01-05,1,1,10,5,5,5,1,2013,1


In [8]:
df = pd.get_dummies(train, columns = ['store','item','dayofmonth','dayofweek','month','weekofyear'])

In [9]:
df.head()

,date,sales,dayofyear,year,store_1,store_2,store_3,store_4,store_5,store_6,...,weekofyear_44,weekofyear_45,weekofyear_46,weekofyear_47,weekofyear_48,weekofyear_49,weekofyear_50,weekofyear_51,weekofyear_52,weekofyear_53
0,2013-01-01,13,1,2013,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2013-01-02,11,2,2013,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2013-01-03,14,3,2013,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2013-01-04,13,4,2013,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2013-01-05,10,5,2013,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_val = df[~msk]
print("train shape: ",df_train.shape)
print("validation shape :",df_val.shape)

train shape:  (730758, 167)
validation shape : (182242, 167)


In [11]:
df_train.drop('date',axis=1,inplace=True)
df_val.drop('date',axis=1,inplace=True)
y_train = df_train['sales'].values
y_test = df_val['sales'].values
X_train = df_train.drop('sales', axis=1).values
X_test = df_val.drop('sales', axis=1).values

/home/swati/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {'task':'train', 'boosting_type':'gbdt', 'objective':'regression', 
              'metric': {'rmse'}, 'num_leaves': 10, 'learning_rate': 0.05, 
              'feature_fraction': 0.8, 'max_depth': 5, 'verbose': 0, 
              'num_boost_round':15000, 'early_stopping_rounds':100, 'nthread':-1}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)


/home/swati/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/swati/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 28.529
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 28.2999
[3]	valid_0's rmse: 28.1156
[4]	valid_0's rmse: 27.9005
[5]	valid_0's rmse: 27.7162
[6]	valid_0's rmse: 27.5285
[7]	valid_0's rmse: 27.3318
[8]	valid_0's rmse: 27.1628
[9]	valid_0's rmse: 26.9996
[10]	valid_0's rmse: 26.8396
[11]	valid_0's rmse: 26.6727
[12]	valid_0's rmse: 26.4998
[13]	valid_0's rmse: 26.3241
[14]	valid_0's rmse: 26.1828
[15]	valid_0's rmse: 26.0134
[16]	valid_0's rmse: 25.8677
[17]	valid_0's rmse: 25.7029
[18]	valid_0's rmse: 25.5645
[19]	valid_0's rmse: 25.4086
[20]	valid_0's rmse: 25.2772
[21]	valid_0's rmse: 25.1306
[22]	valid_0's rmse: 25.0017
[23]	valid_0's rmse: 24.8572
[24]	valid_0's rmse: 24.7295
[25]	valid_0's rmse: 24.5924
[26]	valid_0's rmse: 24.4643
[27]	valid_0's rmse: 24.3285
[28]	valid_0's rmse: 24.1891
[29]	valid_0's rmse: 24.0512
[30]	valid_0's rmse: 23.921
[31]	valid_0's rmse: 23.7873
[32]	valid_0's rmse: 23.6818
[33]	valid_0's rmse:

[277]	valid_0's rmse: 11.1362
[278]	valid_0's rmse: 11.1199
[279]	valid_0's rmse: 11.1029
[280]	valid_0's rmse: 11.0866
[281]	valid_0's rmse: 11.0695
[282]	valid_0's rmse: 11.053
[283]	valid_0's rmse: 11.0362
[284]	valid_0's rmse: 11.0174
[285]	valid_0's rmse: 11.0029
[286]	valid_0's rmse: 10.9879
[287]	valid_0's rmse: 10.9734
[288]	valid_0's rmse: 10.9554
[289]	valid_0's rmse: 10.9388
[290]	valid_0's rmse: 10.9226
[291]	valid_0's rmse: 10.9073
[292]	valid_0's rmse: 10.8939
[293]	valid_0's rmse: 10.8792
[294]	valid_0's rmse: 10.8634
[295]	valid_0's rmse: 10.8494
[296]	valid_0's rmse: 10.835
[297]	valid_0's rmse: 10.8193
[298]	valid_0's rmse: 10.8024
[299]	valid_0's rmse: 10.7877
[300]	valid_0's rmse: 10.7721
[301]	valid_0's rmse: 10.7581
[302]	valid_0's rmse: 10.7447
[303]	valid_0's rmse: 10.7297
[304]	valid_0's rmse: 10.7158
[305]	valid_0's rmse: 10.703
[306]	valid_0's rmse: 10.6881
[307]	valid_0's rmse: 10.6737
[308]	valid_0's rmse: 10.6581
[309]	valid_0's rmse: 10.6441
[310]	valid_0

[559]	valid_0's rmse: 8.99923
[560]	valid_0's rmse: 8.99661
[561]	valid_0's rmse: 8.99454
[562]	valid_0's rmse: 8.99201
[563]	valid_0's rmse: 8.98926
[564]	valid_0's rmse: 8.98637
[565]	valid_0's rmse: 8.98402
[566]	valid_0's rmse: 8.9812
[567]	valid_0's rmse: 8.97862
[568]	valid_0's rmse: 8.97579
[569]	valid_0's rmse: 8.97328
[570]	valid_0's rmse: 8.97081
[571]	valid_0's rmse: 8.96833
[572]	valid_0's rmse: 8.96486
[573]	valid_0's rmse: 8.96193
[574]	valid_0's rmse: 8.95986
[575]	valid_0's rmse: 8.9575
[576]	valid_0's rmse: 8.95526
[577]	valid_0's rmse: 8.95294
[578]	valid_0's rmse: 8.95024
[579]	valid_0's rmse: 8.94755
[580]	valid_0's rmse: 8.94556
[581]	valid_0's rmse: 8.943
[582]	valid_0's rmse: 8.94059
[583]	valid_0's rmse: 8.93843
[584]	valid_0's rmse: 8.93583
[585]	valid_0's rmse: 8.93366
[586]	valid_0's rmse: 8.93165
[587]	valid_0's rmse: 8.92919
[588]	valid_0's rmse: 8.92703
[589]	valid_0's rmse: 8.92508
[590]	valid_0's rmse: 8.92261
[591]	valid_0's rmse: 8.92018
[592]	valid_0'

[833]	valid_0's rmse: 8.55194
[834]	valid_0's rmse: 8.55148
[835]	valid_0's rmse: 8.55051
[836]	valid_0's rmse: 8.54961
[837]	valid_0's rmse: 8.54817
[838]	valid_0's rmse: 8.54739
[839]	valid_0's rmse: 8.5463
[840]	valid_0's rmse: 8.54532
[841]	valid_0's rmse: 8.5445
[842]	valid_0's rmse: 8.54341
[843]	valid_0's rmse: 8.5427
[844]	valid_0's rmse: 8.54139
[845]	valid_0's rmse: 8.54048
[846]	valid_0's rmse: 8.53942
[847]	valid_0's rmse: 8.5385
[848]	valid_0's rmse: 8.5377
[849]	valid_0's rmse: 8.5362
[850]	valid_0's rmse: 8.53511
[851]	valid_0's rmse: 8.5341
[852]	valid_0's rmse: 8.53358
[853]	valid_0's rmse: 8.53181
[854]	valid_0's rmse: 8.53085
[855]	valid_0's rmse: 8.53005
[856]	valid_0's rmse: 8.5291
[857]	valid_0's rmse: 8.5279
[858]	valid_0's rmse: 8.52718
[859]	valid_0's rmse: 8.52662
[860]	valid_0's rmse: 8.52541
[861]	valid_0's rmse: 8.52392
[862]	valid_0's rmse: 8.52308
[863]	valid_0's rmse: 8.5222
[864]	valid_0's rmse: 8.521
[865]	valid_0's rmse: 8.5201
[866]	valid_0's rmse: 8

...

[10322]	valid_0's rmse: 7.2957
[10323]	valid_0's rmse: 7.2957
[10324]	valid_0's rmse: 7.29571
[10325]	valid_0's rmse: 7.2957
[10326]	valid_0's rmse: 7.29569
[10327]	valid_0's rmse: 7.2957
[10328]	valid_0's rmse: 7.2957
[10329]	valid_0's rmse: 7.2957
[10330]	valid_0's rmse: 7.2957
[10331]	valid_0's rmse: 7.2957
[10332]	valid_0's rmse: 7.29571
[10333]	valid_0's rmse: 7.29568
[10334]	valid_0's rmse: 7.29567
[10335]	valid_0's rmse: 7.29566
[10336]	valid_0's rmse: 7.29566
[10337]	valid_0's rmse: 7.29566
[10338]	valid_0's rmse: 7.29567
[10339]	valid_0's rmse: 7.29567
[10340]	valid_0's rmse: 7.29567
[10341]	valid_0's rmse: 7.29565
[10342]	valid_0's rmse: 7.29566
[10343]	valid_0's rmse: 7.29566
[10344]	valid_0's rmse: 7.29566
[10345]	valid_0's rmse: 7.29566
[10346]	valid_0's rmse: 7.29566
[10347]	valid_0's rmse: 7.29567
[10348]	valid_0's rmse: 7.29567
[10349]	valid_0's rmse: 7.29567
[10350]	valid_0's rmse: 7.29566
[10351]	valid_0's rmse: 7.29566
[10352]	valid_0's rmse: 7.29566
[10353]	valid_0'

[10589]	valid_0's rmse: 7.29532
[10590]	valid_0's rmse: 7.29531
[10591]	valid_0's rmse: 7.29531
[10592]	valid_0's rmse: 7.29531
[10593]	valid_0's rmse: 7.29531
[10594]	valid_0's rmse: 7.2953
[10595]	valid_0's rmse: 7.2953
[10596]	valid_0's rmse: 7.2953
[10597]	valid_0's rmse: 7.29529
[10598]	valid_0's rmse: 7.29529
[10599]	valid_0's rmse: 7.29528
[10600]	valid_0's rmse: 7.29529
[10601]	valid_0's rmse: 7.29529
[10602]	valid_0's rmse: 7.2953
[10603]	valid_0's rmse: 7.2953
[10604]	valid_0's rmse: 7.29529
[10605]	valid_0's rmse: 7.2953
[10606]	valid_0's rmse: 7.29531
[10607]	valid_0's rmse: 7.2953
[10608]	valid_0's rmse: 7.29531
[10609]	valid_0's rmse: 7.29531
[10610]	valid_0's rmse: 7.2953
[10611]	valid_0's rmse: 7.2953
[10612]	valid_0's rmse: 7.2953
[10613]	valid_0's rmse: 7.29529
[10614]	valid_0's rmse: 7.2953
[10615]	valid_0's rmse: 7.2953
[10616]	valid_0's rmse: 7.2953
[10617]	valid_0's rmse: 7.29528
[10618]	valid_0's rmse: 7.29528
[10619]	valid_0's rmse: 7.29528
[10620]	valid_0's rms

[10849]	valid_0's rmse: 7.29507
[10850]	valid_0's rmse: 7.29506
[10851]	valid_0's rmse: 7.29505
[10852]	valid_0's rmse: 7.29504
[10853]	valid_0's rmse: 7.29502
[10854]	valid_0's rmse: 7.29501
[10855]	valid_0's rmse: 7.295
[10856]	valid_0's rmse: 7.295
[10857]	valid_0's rmse: 7.29499
[10858]	valid_0's rmse: 7.29498
[10859]	valid_0's rmse: 7.29497
[10860]	valid_0's rmse: 7.29496
[10861]	valid_0's rmse: 7.29496
[10862]	valid_0's rmse: 7.29495
[10863]	valid_0's rmse: 7.29494
[10864]	valid_0's rmse: 7.29494
[10865]	valid_0's rmse: 7.29493
[10866]	valid_0's rmse: 7.29492
[10867]	valid_0's rmse: 7.29492
[10868]	valid_0's rmse: 7.29491
[10869]	valid_0's rmse: 7.29491
[10870]	valid_0's rmse: 7.29491
[10871]	valid_0's rmse: 7.2949
[10872]	valid_0's rmse: 7.2949
[10873]	valid_0's rmse: 7.2949
[10874]	valid_0's rmse: 7.2949
[10875]	valid_0's rmse: 7.29489
[10876]	valid_0's rmse: 7.29489
[10877]	valid_0's rmse: 7.29489
[10878]	valid_0's rmse: 7.29488
[10879]	valid_0's rmse: 7.29486
[10880]	valid_0'

[11107]	valid_0's rmse: 7.29425
[11108]	valid_0's rmse: 7.29425
[11109]	valid_0's rmse: 7.29425
[11110]	valid_0's rmse: 7.29425
[11111]	valid_0's rmse: 7.29426
[11112]	valid_0's rmse: 7.29425
[11113]	valid_0's rmse: 7.29425
[11114]	valid_0's rmse: 7.29425
[11115]	valid_0's rmse: 7.29424
[11116]	valid_0's rmse: 7.29424
[11117]	valid_0's rmse: 7.29423
[11118]	valid_0's rmse: 7.29422
[11119]	valid_0's rmse: 7.29421
[11120]	valid_0's rmse: 7.29421
[11121]	valid_0's rmse: 7.2942
[11122]	valid_0's rmse: 7.2942
[11123]	valid_0's rmse: 7.29419
[11124]	valid_0's rmse: 7.2942
[11125]	valid_0's rmse: 7.2942
[11126]	valid_0's rmse: 7.29419
[11127]	valid_0's rmse: 7.29419
[11128]	valid_0's rmse: 7.2942
[11129]	valid_0's rmse: 7.2942
[11130]	valid_0's rmse: 7.2942
[11131]	valid_0's rmse: 7.2942
[11132]	valid_0's rmse: 7.2942
[11133]	valid_0's rmse: 7.2942
[11134]	valid_0's rmse: 7.29419
[11135]	valid_0's rmse: 7.29419
[11136]	valid_0's rmse: 7.29419
[11137]	valid_0's rmse: 7.2942
[11138]	valid_0's r

[11371]	valid_0's rmse: 7.29376
[11372]	valid_0's rmse: 7.29377
[11373]	valid_0's rmse: 7.29377
[11374]	valid_0's rmse: 7.29377
[11375]	valid_0's rmse: 7.29377
[11376]	valid_0's rmse: 7.29377
[11377]	valid_0's rmse: 7.29377
[11378]	valid_0's rmse: 7.29377
[11379]	valid_0's rmse: 7.29376
[11380]	valid_0's rmse: 7.29376
[11381]	valid_0's rmse: 7.29375
[11382]	valid_0's rmse: 7.29377
[11383]	valid_0's rmse: 7.29377
[11384]	valid_0's rmse: 7.29377
[11385]	valid_0's rmse: 7.29377
[11386]	valid_0's rmse: 7.29378
[11387]	valid_0's rmse: 7.29377
[11388]	valid_0's rmse: 7.29378
[11389]	valid_0's rmse: 7.29378
[11390]	valid_0's rmse: 7.29377
[11391]	valid_0's rmse: 7.29377
[11392]	valid_0's rmse: 7.29377
[11393]	valid_0's rmse: 7.29377
[11394]	valid_0's rmse: 7.29376
[11395]	valid_0's rmse: 7.29375
[11396]	valid_0's rmse: 7.29375
[11397]	valid_0's rmse: 7.29375
[11398]	valid_0's rmse: 7.29374
[11399]	valid_0's rmse: 7.29373
[11400]	valid_0's rmse: 7.29373
[11401]	valid_0's rmse: 7.29372
[11402]	

In [14]:
test['dayofmonth'] = test.date.dt.day
test['dayofyear'] = test.date.dt.dayofyear
test['dayofweek'] = test.date.dt.dayofweek
test['month'] = test.date.dt.month
test['year'] = test.date.dt.year
test['weekofyear'] = test.date.dt.weekofyear

In [28]:
df_test = pd.get_dummies(test, columns = ['store','item','dayofmonth','dayofweek','month','weekofyear'])
df_test.drop(['id','date'],axis=1,inplace=True)
X_prediction = df_test.values
# X_prediction
df_test.head()

,dayofyear,year,store_1,store_2,store_3,store_4,store_5,store_6,store_7,store_8,...,weekofyear_4,weekofyear_5,weekofyear_6,weekofyear_7,weekofyear_8,weekofyear_9,weekofyear_10,weekofyear_11,weekofyear_12,weekofyear_13
0,1,2018,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2018,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2018,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,2018,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2018,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
predictions = gbm.predict(X_prediction,num_iteration = gbm.best_iteration)

In [31]:
sample_sub['sales']= predictions

In [32]:
sample_sub

,id,sales
0,0,12.582610
1,1,13.548208
2,2,14.299131
3,3,14.627948
4,4,16.525432
5,5,17.698905
6,6,17.056383
7,7,12.402573
8,8,13.650310
9,9,14.062369
